In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm
import spacy
import pandas as pd
import re
nlp = spacy.load("en_core_web_sm")

In [ ]:
def add_ing_to_verb(verb):
    if verb.endswith('e') and not verb.endswith('ee') and verb != 'be': # correct -ing ending
        return verb[:-1] + 'ing'
    elif verb.endswith(('ie')):
        return verb[:-2] + 'ying'
    else:
        return verb + 'ing'

def add_s_to_verb(verb):
    if verb.endswith("y") and not verb[-2] in "aeiou":
        return verb[:-1] + "ies"
    elif verb.endswith(("o", "ch", "s", "sh", "x", "z")):
        return verb + "es"
    else:
        return verb + "s"

def conjugate_verb(verb, subject_token, target_tense):
    is_plural = subject_token.tag_ in ["NNS", "NNPS"] if subject_token else False

    if target_tense == "present_simple":
        if not is_plural:
            return add_s_to_verb(verb)
        else:
            return verb
    elif target_tense == "future_simple":
        return f"will {verb}"
    elif target_tense == "past_continuous":
        return f"{'were' if is_plural else 'was'} {add_ing_to_verb(verb)}"

def transform_tense(sentence, target_tense):
    doc = nlp(sentence)
    new_sentence = []
    subject = None

    for token in doc:
        if token.dep_ == "nsubj" or token.dep_ == "nsubjpass":
            subject = token
        if token.pos_ == "VERB":
            new_verb = conjugate_verb(token.lemma_, subject, target_tense) if target_tense != "" else token.text
            new_sentence.append(new_verb)
        else:
            new_sentence.append(token.text)
        if token.dep_ == "punct" and token.text in [".", "!", "?"]:
            subject = None

    processed_sentence = " ".join(new_sentence)
    processed_sentence = re.sub(r'\s+([?.!,])', r'\1', processed_sentence)  # remove spaces before punctuation
    return processed_sentence

def apply_tense_transformations(df):
    updated_sentences = []
    target_tenses = ["", "present_simple", "future_simple", "past_continuous"]

    for index, row in df.iterrows():
        target_tense = target_tenses[index % len(target_tenses)]
        transformed_sentence = transform_tense(row['premise'], target_tense)
        updated_sentences.append(transformed_sentence)

    df['premise'] = updated_sentences
    return df

In [ ]:
df = pd.read_csv('2 premises_plural.csv')
updated_df = apply_tense_transformations(df)
updated_df.to_csv('3 premises_tenses.csv', index=False)